In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import gc
import random

import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings(action='ignore')

from pycaret.classification import *

import random as python_random
seed = 42
np.random.seed(seed)
python_random.seed(seed)
set_config('seed', seed)

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [60]:
def chg_qua(x):
    if x == 0:
        return 0
    else:
        return 1

In [26]:
train_err['date'] = train_err['time'].astype(str).str.slice(0,8)
train_err['model_fwver'] = train_err.model_nm + train_err.fwver
train_err['errtype_code'] = train_err.errtype.astype(str) + train_err.errcode

train_err_0 = train_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(train_err_0.head())

test_err['date'] = test_err['time'].astype(str).str.slice(0,8)
test_err['model_fwver'] = test_err.model_nm + test_err.fwver
test_err['errtype_code'] = test_err.errtype.astype(str) + test_err.errcode

test_err_0 = test_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(test_err_0.head())

,user_id,date,model_fwver,errtype_code
0,10000,20201101,model_305.15.2138,151
1,10000,20201101,model_305.15.2138,121
2,10000,20201101,model_305.15.2138,111
3,10000,20201101,model_305.15.2138,161
4,10000,20201101,model_305.15.2138,40


,user_id,date,model_fwver,errtype_code
0,30000,20201101,model_104.16.3553,311
1,30000,20201101,model_104.16.3553,332
2,30000,20201101,model_104.16.3553,151
3,30000,20201101,model_104.16.3553,221
4,30000,20201101,model_104.16.3553,111


In [ ]:
train_err_1 = train_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
train_err_1.columns = ['user_id','date','date_cnt']
train_err_1 = train_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
train_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(train_err_1.head())

test_err_1 = test_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
test_err_1.columns = ['user_id','date','date_cnt']
test_err_1 = test_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
test_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(test_err_1.head())

train_err_1.shape

In [ ]:
train = train_err_1.copy()
test = test_err_1.copy()

In [ ]:
train_model = set(train_err.model_fwver.unique())
test_model = set(test_err.model_fwver.unique())
models = train_model & test_model
models = list(models)

for model in models:
    train[model] = 0
    test[model] = 0
    
train.shape, test.shape    

In [ ]:
train_err_2 = train_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
train_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
display(train_err_2.head())

test_err_2 = test_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
test_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
display(test_err_2.head())

In [ ]:
for idx, col in tqdm(enumerate(train_err_2.values)):
    i = train[train.user_id == train_err_2.loc[idx].user_id].index
    c = train.columns[train.columns == train_err_2.loc[idx].model_fwver]
    
    train.loc[i, c] += train_err_2.loc[idx].model_fwver_cnt    

for idx, col in tqdm(enumerate(test_err_2.values)):
    i = test[test.user_id == test_err_2.loc[idx].user_id].index
    c = test.columns[test.columns == test_err_2.loc[idx].model_fwver]
    
    test.loc[i, c] += test_err_2.loc[idx].model_fwver_cnt    

In [ ]:
train_error = set(train_err.errtype.unique())
test_error = set(test_err.errtype.unique())
errors = train_error & test_error
errors = list(errors)

for error in errors:
    train['E'+str(error)] = 0
    test['E'+str(error)] = 0
    
train.shape, test.shape    

In [ ]:
train_err_3 = train_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
train_err_3.columns = ['user_id','errtype','errtype_cnt']
display(train_err_3.head())

test_err_3 = test_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
test_err_3.columns = ['user_id','errtype','errtype_cnt']
display(test_err_3.head())

In [ ]:
for idx, col in tqdm(enumerate(train_err_3.values)):
    i = train[train.user_id == train_err_3.loc[idx].user_id].index
    c = train.columns[train.columns == 'E'+str(train_err_3.loc[idx].errtype)]
    
    train.loc[i, c] += train_err_3.loc[idx].errtype_cnt    

for idx, col in tqdm(enumerate(test_err_3.values)):
    i = test[test.user_id == test_err_3.loc[idx].user_id].index
    c = test.columns[test.columns == 'E'+str(test_err_3.loc[idx].errtype)]
    
    test.loc[i, c] += test_err_3.loc[idx].errtype_cnt    

In [ ]:
train_qua_0 = train_qua.drop_duplicates()
test_qua_0 = test_qua.drop_duplicates()

train_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)
test_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)

train_qua_0.fillna(0, inplace=True)
test_qua_0.fillna(0, inplace=True)

In [ ]:
cols = ['quality_0', 'quality_1', 'quality_2', 'quality_5','quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']

for col in cols:
    train_qua_0[col] = train_qua_0[col].apply(chg_qua)
    test_qua_0[col] = test_qua_0[col].apply(chg_qua)

In [ ]:
train_qua_1 = train_qua_0.groupby('user_id').sum().reset_index()
test_qua_1 = test_qua_0.groupby('user_id').sum().reset_index()

In [ ]:
train = train.merge(train_qua_1, on='user_id', how='left').fillna(0)
test = test.merge(test_qua_1, on='user_id', how='left').fillna(0)

In [6]:
train.to_csv("./train.csv", index=False)
test.to_csv("./test.csv", index=False)

In [68]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [69]:
display(train.head())
display(test.head())

train.shape, test.shape

,user_id,date_cnt,date_sum,model_403.11.1167,model_804.73.2571,model_004.22.1750,model_610,model_68.5.3,model_204.33.1185,model_705.66.3237,...,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,30,317,0,0,0,0,0,0,0,...,0,0,4,0,4,0,4,4,0,0
1,10001,30,2365,0,0,0,0,0,379,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,29,306,0,0,0,0,0,0,0,...,1,2,22,4,22,0,22,22,1,0
3,10003,30,306,0,0,0,0,0,81,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,30,777,0,0,645,0,0,0,0,...,1,1,6,2,6,0,6,6,1,0


,user_id,date_cnt,date_sum,model_403.11.1167,model_804.73.2571,model_004.22.1750,model_610,model_68.5.3,model_204.33.1185,model_705.66.3237,...,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,29,2750,0,0,0,0,0,0,0,...,0,0,2,1,2,0,2,2,0,0
1,30001,28,284,0,0,0,0,0,0,0,...,1,1,10,1,10,0,10,10,1,0
2,30002,30,941,0,0,733,0,0,0,0,...,3,3,26,3,26,0,26,26,3,0
3,30003,28,371,0,0,246,0,0,0,0,...,0,0,13,5,13,0,13,13,0,0
4,30004,30,881,0,0,0,0,0,0,0,...,1,1,5,3,5,0,5,5,1,0


((15000, 86), (14998, 86))

In [24]:
train.corr()['prob'].sort_values(ascending=False)[:10]

prob        1.000000
E20         0.287503
E18         0.259057
E16         0.212896
E15         0.211715
E37         0.177654
E36         0.177015
E40         0.147105
E26         0.142512
date_cnt    0.135163
Name: prob, dtype: float64

In [66]:
train_err_d1 = train_err.groupby(['user_id','date']).count().groupby('user_id')['time'].max().reset_index()
train_err_d1.columns = ['user_id', 'day_max']
train_err_d1

,user_id,day_max
0,10000,20
1,10001,1452
2,10002,17
3,10003,24
4,10004,102
...,...,...
14995,24995,30
14996,24996,4
14997,24997,64
14998,24998,37


In [39]:
test_err_d1 = test_err.groupby(['user_id','date']).count().groupby('user_id')['time'].max().reset_index()
test_err_d1.columns = ['user_id', 'day_max']
test_err_d1

,user_id,day_max
0,30000,808
1,30001,22
2,30002,67
3,30003,75
4,30004,85
...,...,...
14993,44994,90
14994,44995,35
14995,44996,430
14996,44997,2606


In [70]:
train = train.merge(train_err_d1, on='user_id', how='left')
test = test.merge(test_err_d1, on='user_id', how='left')
train

,user_id,date_cnt,date_sum,model_403.11.1167,model_804.73.2571,model_004.22.1750,model_610,model_68.5.3,model_204.33.1185,model_705.66.3237,...,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,day_max
0,10000,30,317,0,0,0,0,0,0,0,...,0,4,0,4,0,4,4,0,0,20
1,10001,30,2365,0,0,0,0,0,379,0,...,0,0,0,0,0,0,0,0,0,1452
2,10002,29,306,0,0,0,0,0,0,0,...,2,22,4,22,0,22,22,1,0,17
3,10003,30,306,0,0,0,0,0,81,0,...,0,0,0,0,0,0,0,0,0,24
4,10004,30,777,0,0,645,0,0,0,0,...,1,6,2,6,0,6,6,1,0,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,10,194,0,0,0,0,0,0,0,...,2,6,2,0,0,0,6,2,0,30
14996,24996,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
14997,24997,30,826,0,0,465,0,0,0,0,...,1,8,1,0,0,0,8,1,0,64
14998,24998,21,155,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,37


In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train_qua_1.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
test.columns

In [ ]:
train

In [ ]:
test.info()

In [ ]:
cols = test.columns[1:]
cols = list(cols)

In [ ]:
min = train[cols].min()
max  = train[cols].max()

In [ ]:
for i, col in enumerate(cols):
    train[col] = (train[col] - min[i]) / (max[i] - min[i])
    test[col] = (test[col] - min[i]) / (max[i] - min[i])

In [61]:
train_prob1 = train_prob.groupby('user_id').count().reset_index()
train_prob1.time = train_prob1.time.apply(chg_qua)
train_prob1.columns = ['user_id', 'prob']
train_prob1

,user_id,prob
0,10001,1
1,10004,1
2,10005,1
3,10006,1
4,10008,1
...,...,...
4995,24983,1
4996,24984,1
4997,24990,1
4998,24997,1


In [71]:
train = train.merge(train_prob1, on='user_id', how='left').fillna(0)
train

,user_id,date_cnt,date_sum,model_403.11.1167,model_804.73.2571,model_004.22.1750,model_610,model_68.5.3,model_204.33.1185,model_705.66.3237,...,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,day_max,prob
0,10000,30,317,0,0,0,0,0,0,0,...,4,0,4,0,4,4,0,0,20,0.0
1,10001,30,2365,0,0,0,0,0,379,0,...,0,0,0,0,0,0,0,0,1452,1.0
2,10002,29,306,0,0,0,0,0,0,0,...,22,4,22,0,22,22,1,0,17,0.0
3,10003,30,306,0,0,0,0,0,81,0,...,0,0,0,0,0,0,0,0,24,0.0
4,10004,30,777,0,0,645,0,0,0,0,...,6,2,6,0,6,6,1,0,102,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,10,194,0,0,0,0,0,0,0,...,6,2,0,0,0,6,2,0,30,0.0
14996,24996,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.0
14997,24997,30,826,0,0,465,0,0,0,0,...,8,1,0,0,0,8,1,0,64,1.0
14998,24998,21,155,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,37,1.0


In [72]:
train.prob = train.prob.astype(int)

In [73]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 88 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   user_id            15000 non-null  int64
 1   date_cnt           15000 non-null  int64
 2   date_sum           15000 non-null  int64
 3   model_403.11.1167  15000 non-null  int64
 4   model_804.73.2571  15000 non-null  int64
 5   model_004.22.1750  15000 non-null  int64
 6   model_610          15000 non-null  int64
 7   model_68.5.3       15000 non-null  int64
 8   model_204.33.1185  15000 non-null  int64
 9   model_705.66.3237  15000 non-null  int64
 10  model_104.16.3571  15000 non-null  int64
 11  model_004.22.1778  15000 non-null  int64
 12  model_104.16.3569  15000 non-null  int64
 13  model_204.33.1171  15000 non-null  int64
 14  model_305.15.2120  15000 non-null  int64
 15  model_504.82.1684  15000 non-null  int64
 16  model_305.15.3104  15000 non-null  int64
 17  model_504.82

In [74]:
train.columns[1:-1]

Index(['date_cnt', 'date_sum', 'model_403.11.1167', 'model_804.73.2571',
       'model_004.22.1750', 'model_610', 'model_68.5.3', 'model_204.33.1185',
       'model_705.66.3237', 'model_104.16.3571', 'model_004.22.1778',
       'model_104.16.3569', 'model_204.33.1171', 'model_305.15.2120',
       'model_504.82.1684', 'model_305.15.3104', 'model_504.82.1778',
       'model_104.16.3553', 'model_504.82.1730', 'model_403.11.1149',
       'model_104.16.3439', 'model_305.15.2092', 'model_305.15.2138',
       'model_004.22.1656', 'model_204.33.1125', 'model_204.33.1261',
       'model_004.22.1666', 'model_403.11.1141', 'model_705.66.3571',
       'model_305.15.2114', 'model_204.33.1149', 'model_804.73.2237',
       'model_004.22.1684', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8',
       'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18',
       'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28',
       'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37

In [75]:
%%time
clf = setup(session_id=seed, data=train, target='prob'
           , numeric_features=train.columns[1:-1])

Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(15000, 88)"
4,Missing Values,False
5,Numeric Features,87
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


Wall time: 4.88 s


In [76]:
best = compare_models(sort = 'AUC', n_select = 5
                     ,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'ridge','qda']
                     )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.7949,0.8169,0.5100,0.8035,0.6238,0.4923,0.5165,11.9409
1,Gradient Boosting Classifier,0.7902,0.8154,0.4791,0.8155,0.6034,0.4736,0.5046,3.5059
2,Light Gradient Boosting Machine,0.7903,0.8123,0.5157,0.7810,0.6211,0.4844,0.5044,0.5227
3,Extra Trees Classifier,0.7843,0.8012,0.4914,0.7803,0.6030,0.4652,0.4886,0.6327
4,Ada Boost Classifier,0.7776,0.7994,0.4849,0.7611,0.5921,0.4495,0.4712,0.9143
5,Extreme Gradient Boosting,0.7817,0.7978,0.5380,0.7360,0.6214,0.4733,0.4848,1.8797
6,Random Forest Classifier,0.7638,0.7662,0.4583,0.7330,0.5637,0.4135,0.4350,0.1227
7,Linear Discriminant Analysis,0.7507,0.7384,0.3306,0.8084,0.4692,0.3419,0.4003,0.1568


In [77]:
blended = blend_models(estimator_list = best, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7790,0.8088,0.4700,0.7796,0.5865,0.4481,0.4748
1,0.8043,0.8299,0.5229,0.8262,0.6404,0.5151,0.5406
2,0.7910,0.8197,0.4943,0.8028,0.6118,0.4797,0.5061
3,0.8010,0.8338,0.4943,0.8439,0.6234,0.5004,0.5335
4,0.7823,0.7981,0.4886,0.7755,0.5995,0.4604,0.4835
Mean,0.7915,0.8181,0.4940,0.8056,0.6123,0.4807,0.5077
SD,0.0099,0.0132,0.0170,0.0264,0.0187,0.0247,0.0261


In [78]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7932,0.8201,0.492,0.8137,0.6132,0.4835,0.512


In [79]:
%%time
final_model = finalize_model(blended)

Wall time: 3min 18s


In [80]:
test_x = sample_submssion[['user_id']].merge(test, on='user_id', how='left').fillna(0)
test_x.shape

(14999, 87)

In [81]:
predictions = predict_model(final_model, data = test_x)

In [82]:
sample_submssion['problem'] = predictions['Score']
sample_submssion.head()

,user_id,problem
0,30000,0.8468
1,30001,0.2377
2,30002,0.3326
3,30003,0.7651
4,30004,0.7930


In [83]:
sample_submssion.to_csv("./submission/submission_20210129-1.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.8468
1,30001,0.2377
2,30002,0.3326
3,30003,0.7651
4,30004,0.7930
...,...,...
14994,44994,0.3333
14995,44995,0.2818
14996,44996,0.5677
14997,44997,0.7631
